# IR Project 4

## Sentiment Analysis

### Load libraries and data

In [1]:
# Import libraries
import json
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package stopwords to /Users/maga/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Load data
infile = open('tweets.json','rb')
tweets = json.load(infile)
infile.close()

### Preprocess data

In [3]:
# Function to remove stopwords
def remove_stopwords(text, lang): 
    text_wo_stopwords = []
    # remove stopwords   
    if lang == 'en':
        stop_words = set(stopwords.words('english'))
    else:
        stop_words = set(stopwords.words('spanish'))
    text = re.split('\s',text)
    for t in text:
        if t not in stop_words:
            text_wo_stopwords.append(t)
    text_wo_stopwords = ' '.join(text_wo_stopwords)
    return text_wo_stopwords

In [4]:
# Function to preprocess tweets - TO-DO: Clean emojis
def preprocess(raw_tweet,lang):
    text = raw_tweet.lower() # convert to lowercase
    text = re.sub('\n',' ',text) # remove '\n'
    text = re.sub(r"http\S+", "",text) # remove urls
    text = re.sub('#',' ',text) # remove '#' but leave text from hashtag
    text = re.sub('@[a-zA-Z]+',' ', text) # remove mentions
    text = re.sub('^rt ',' ', text) # remove 'rt'
    text = re.sub('[,\.\:\!¡\?\¿\_–-\’\$%]',' ',text) # remove punctuation
    text = re.sub('[0-9]+', ' ',text) # remove numbers
    text = re.sub('\s+',' ',text) # remove extra whitespaces
    text = re.sub('^\s+','',text) # remove space(s) at start
    text = re.sub('\s+$','',text) # remove space(s) at end
    if lang == 'en' or 'es': # no stopwrods for hindi
        text = remove_stopwords(text,lang)
    return text

In [5]:
# Extract tweets per language
data_en, data_es, data_hi = [], [], []

for tweet in tweets:
    text = tweet['tweet_text']
    if tweet['tweet_lang'] == 'en':
        text = preprocess(text,'en')
        data_en.append(text)
    elif tweet['tweet_lang'] == 'es':
        text = preprocess(text,'es')
        data_es.append(text)
    elif tweet['tweet_lang'] == 'hi':
        text = preprocess(text,'hi')
        data_hi.append(text) 

In [6]:
data = pd.DataFrame(data_en)

In [7]:
data.head()

,0
0,bipartisan infrastructure law best roads bridg...
1,congratulations washington spirit winning firs...
2,unions built middle class country gotten infra...
3,last year alone united states extreme weather ...
4,took office got right work getting shots arms ...


### Test different sentiment analyzers

#### NLTK 

In [8]:
nltk.download('vader_lexicon')
sentiments = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/maga/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [10]:
data["Positive"] = [sentiments.polarity_scores(i)["pos"] for i in data[0]]
data["Negative"] = [sentiments.polarity_scores(i)["neg"] for i in data[0]]
data["Neutral"] = [sentiments.polarity_scores(i)["neu"] for i in data[0]]

In [11]:
data.head()

,0,Positive,Negative,Neutral
0,bipartisan infrastructure law best roads bridg...,0.189,0.000,0.811
1,congratulations washington spirit winning firs...,0.645,0.047,0.308
2,unions built middle class country gotten infra...,0.000,0.000,1.000
3,last year alone united states extreme weather ...,0.255,0.160,0.585
4,took office got right work getting shots arms ...,0.123,0.167,0.710


In [12]:
# Look at the total of the sentiment scores
pos = sum(data["Positive"])
neg = sum(data["Negative"])
neu = sum(data["Neutral"])

print("Positive: ", pos)
print("Negative: ", neg)
print("Neutral: ", neu)

Positive:  941.0550000000027
Negative:  313.167
Neutral:  3176.8080000000045


#### TextBlob

In [33]:
#!pip install textblob

     |████████████████████████████████| 636 kB 3.6 MB/s eta 0:00:01


In [13]:
from textblob import TextBlob

#for i, tweet_text in zip(range(20), tweets_text_collection):
for tweet in data_en:
    print(tweet)
    analysis = TextBlob(tweet)
    
    # Analyse the sentiment.
    #print(analysis.sentiment) # returns polarity and subjectivity
    print(analysis.sentiment_assessments) # returns polarity, subjectivity and assessments
    
    # Polarity is a value between [-1.0, 1.0] 
    # Subjectivity is within the range [0.0, 1.0] where 0.0 is very objective and 1.0 is very subjective.
    # Assessments is a list of polarity and subjectivity scores for the assessed tokens.
    
    print("-"*20)

bipartisan infrastructure law best roads bridges ports airports lead world st century modern cars trucks transit systems building moving
Sentiment(polarity=0.6, subjectivity=0.3, assessments=[(['best'], 1.0, 0.3, None), (['modern'], 0.2, 0.3, None)])
--------------------
congratulations washington spirit winning first-ever nwsl championship despite year shortage challenges made district country proud
Sentiment(polarity=0.65, subjectivity=0.875, assessments=[(['winning'], 0.5, 0.75, None), (['proud'], 0.8, 1.0, None)])
--------------------
unions built middle class country gotten infrastructure law done without
Sentiment(polarity=0.0, subjectivity=0.0, assessments=[(['middle'], 0.0, 0.0, None)])
--------------------
last year alone united states extreme weather events caused billion damage bipartisan infrastructure law builds back bridges water systems power lines levees better stronger
Sentiment(polarity=0.09375, subjectivity=0.39166666666666666, assessments=[(['last'], 0.0, 0.06666666

years september commemorate lives lost honor risked gave lives saw days followed unity greatest strength makes us forget
Sentiment(polarity=1.0, subjectivity=1.0, assessments=[(['greatest'], 1.0, 1.0, None)])
--------------------
morning first lady stopped brookland middle school washington c students head back school around country administration committed help keep safe covid- classroom
Sentiment(polarity=0.1875, subjectivity=0.20833333333333331, assessments=[(['first'], 0.25, 0.3333333333333333, None), (['middle'], 0.0, 0.0, None), (['back'], 0.0, 0.0, None), (['safe'], 0.5, 0.5, None)])
--------------------
message unvaccinated americans wait need see made vaccinations free safe convenient vaccine fda approval million americans gotten least one shot right thing
Sentiment(polarity=0.22142857142857145, subjectivity=0.5589285714285714, assessments=[(['free'], 0.4, 0.8, None), (['safe'], 0.5, 0.5, None), (['least'], -0.3, 0.4, None), (['right'], 0.2857142857142857, 0.5357142857142857, 

Sentiment(polarity=0.0, subjectivity=0.0, assessments=[])
--------------------
honored welcome excellency mustafa al-kadhimi prime minister republic iraq ahead bipartisan meeting yesterday discussed close partnership upcoming iraqi elections &amp; advancing security health economic development governance &amp; climate priorities
Sentiment(polarity=0.5, subjectivity=0.55, assessments=[(['welcome'], 0.8, 0.9, None), (['economic'], 0.2, 0.2, None)])
--------------------
middle class aspire backbone democracy lift working families strengthen democracy address root causes inequality ensure economy works americans
Sentiment(polarity=0.0, subjectivity=0.0, assessments=[(['middle'], 0.0, 0.0, None)])
--------------------
today joined first hearing economic inequality growing years intensified widening chasm ceo compensation &amp; worker pay gone unfair immoral must study &amp; combat crisis
Sentiment(polarity=-0.016666666666666663, subjectivity=0.5111111111111111, assessments=[(['first'], 0.25

Sentiment(polarity=0.1, subjectivity=0.225, assessments=[(['equally'], 0.0, 0.25, None), (['economic'], 0.2, 0.2, None)])
--------------------
pleasure join us today g parliament conversation advancing environmental racial economic justice worldwide togetheronclimate
Sentiment(polarity=0.2, subjectivity=0.2, assessments=[(['economic'], 0.2, 0.2, None)])
--------------------
solve challenges facing global community coordinated international response speaker house honor host th annual g speakers meeting g parliament togetheronclimate
Sentiment(polarity=0.0, subjectivity=0.0, assessments=[(['global'], 0.0, 0.0, None), (['international'], 0.0, 0.0, None)])
--------------------
privilege host year g parliament meeting behalf united states colleagues come together discuss confronting coronavirus standing togetheronclimate securing economic &amp; environmental justice
Sentiment(polarity=0.2, subjectivity=0.2, assessments=[(['economic'], 0.2, 0.2, None)])
--------------------
minutes left kick

biden america back open lawlessness border
Sentiment(polarity=0.0, subjectivity=0.25, assessments=[(['back'], 0.0, 0.0, None), (['open'], 0.0, 0.5, None)])
--------------------
one week ago biden dhs secretary said “no” crisis border… foxnews
Sentiment(polarity=0.0, subjectivity=0.0, assessments=[])
--------------------
release cruz barrasso colleagues biden admin must impose sanctions russia nord stream pipeline
Sentiment(polarity=0.0, subjectivity=0.0, assessments=[])
--------------------
completed putin's pipeline would make european allies &amp; partners susceptible moscow's coercion &amp; malign influence colleagues &amp; urging biden admin implement additional sanctions immediately prevent pipeline coming online
Sentiment(polarity=0.0, subjectivity=0.0, assessments=[(['european'], 0.0, 0.0, None)])
--------------------
tdem continues working provide damage data submitted us texans impacted winterstorm will…
Sentiment(polarity=0.0, subjectivity=0.0, assessments=[])
---------------

--------------------
september national forecasts predict new covid deaths likely reported week ending october would bring projected total number u deaths
Sentiment(polarity=0.04545454545454545, subjectivity=0.7348484848484849, assessments=[(['new'], 0.13636363636363635, 0.45454545454545453, None), (['likely'], 0.0, 1.0, None), (['total'], 0.0, 0.75, None)])
--------------------
september national forecasts predict new covid hospitalizations likely reported october
Sentiment(polarity=0.06818181818181818, subjectivity=0.7272727272727273, assessments=[(['new'], 0.13636363636363635, 0.45454545454545453, None), (['likely'], 0.0, 1.0, None)])
--------------------
every covid cases mid-june oct contact tracing reduced spread much prevented cases accounting prevention measures like mask wearing learn
Sentiment(polarity=0.2, subjectivity=0.2, assessments=[(['much'], 0.2, 0.2, None)])
--------------------
partnership we've launched two🆕 elearning courses aimed educating fetp trainees &amp; grad

--------------------
get vaccinated late
Sentiment(polarity=-0.3, subjectivity=0.6, assessments=[(['late'], -0.3, 0.6, None)])
--------------------
get vaccinated safe free effective everyone age protect &amp; community
Sentiment(polarity=0.5, subjectivity=0.7000000000000001, assessments=[(['safe'], 0.5, 0.5, None), (['free'], 0.4, 0.8, None), (['effective'], 0.6, 0.8, None)])
--------------------
get vaccinated
Sentiment(polarity=0.0, subjectivity=0.0, assessments=[])
--------------------
natalie palugyai values &amp; expertise ca champion workers raised immigrant parents natalie brings a…
Sentiment(polarity=0.0, subjectivity=0.0, assessments=[])
--------------------
"i went apply works real paid bills paid months rent plus they're paying til september help get sleep better night " billion rent relief need
Sentiment(polarity=0.35, subjectivity=0.4, assessments=[(['real'], 0.2, 0.30000000000000004, None), (['better'], 0.5, 0.5, None)])
--------------------
one kicked home due covid inv

Sentiment(polarity=0.0, subjectivity=0.0, assessments=[])
--------------------
unite fightcorona largestvaccinedrive tyoharonkerangcabkesang ministry health family welfare wishes everyone happy safe festive season india
Sentiment(polarity=0.65, subjectivity=0.75, assessments=[(['happy'], 0.8, 1.0, None), (['safe'], 0.5, 0.5, None)])
--------------------
खुशियां फैलाएं वायरस नहीं let celebrate festival lights taking necessary covid precautions stay…
Sentiment(polarity=0.0, subjectivity=1.0, assessments=[(['necessary'], 0.0, 1.0, None)])
--------------------
glad see one vaccine covaxin granted emergency use listing products fight covid …
Sentiment(polarity=0.5, subjectivity=1.0, assessments=[(['glad'], 0.5, 1.0, None)])
--------------------
covaxin eul expands availability 💉 effective medical tools end covid assessed wh…
Sentiment(polarity=0.3, subjectivity=0.4, assessments=[(['effective'], 0.6, 0.8, None), (['medical'], 0.0, 0.0, None)])
--------------------
covaxin vaccine also review

Sentiment(polarity=-0.05, subjectivity=0.8500000000000001, assessments=[(['passionate'], -0.05, 0.8500000000000001, None)])
--------------------
two new trains flagged gandhinagar capital varanasi superfast express memu service trains bet…
Sentiment(polarity=0.13636363636363635, subjectivity=0.45454545454545453, assessments=[(['new'], 0.13636363636363635, 0.45454545454545453, None)])
--------------------
railways related projects electrification surendranagar pipavav section ensure easy freight c…
Sentiment(polarity=0.21666666666666667, subjectivity=0.6166666666666667, assessments=[(['related'], 0.0, 0.4, None), (['easy'], 0.43333333333333335, 0.8333333333333334, None)])
--------------------
always wanted railway stations top quality apart travels boost commerce ho…
Sentiment(polarity=0.5, subjectivity=0.5, assessments=[(['top'], 0.5, 0.5, None)])
--------------------
pm tomorrow th july range interesting development works inaugurated gujarat works cove…
Sentiment(polarity=0.5, subject

diwali add sweetness celebration ordering sweets amazing home baker bengaluru my…
Sentiment(polarity=0.6000000000000001, subjectivity=0.9, assessments=[(['amazing'], 0.6000000000000001, 0.9, None)])
--------------------
bought diyas women vendor selling diyaa roadside best part accepti…
Sentiment(polarity=1.0, subjectivity=0.3, assessments=[(['best'], 1.0, 0.3, None)])
--------------------
diwali buy women entrepreneurs wife startup thank initiative mam…
Sentiment(polarity=0.0, subjectivity=0.0, assessments=[])
--------------------
gift packs arrived today home bakery'first icing owner nidhi jain thane maharashtra narisekharidari loved t…
Sentiment(polarity=0.475, subjectivity=0.5666666666666667, assessments=[(['first'], 0.25, 0.3333333333333333, None), (['loved'], 0.7, 0.8, None)])
--------------------
buy local illuminate festival also light house poor brother siste…
Sentiment(polarity=0.0, subjectivity=0.4333333333333333, assessments=[(['local'], 0.0, 0.0, None), (['light'], 0.4, 0.

In [14]:
# We could translate tweets into English and analyze sentiment using the same model
for tweet in data_hi[0:3]:
    print(tweet)
    blob = TextBlob(tweet)
    print(blob.detect_language())
    print(blob.translate(to="en"))

कोरोना के विरुद्ध जागरूकता अभियान में मानव सेवा संस्थान के कार्यकर्ताओं द्वारा शाहबाद ब्लॉक के ग्राम बडारा में टीकाकरण…
hi
In the awareness campaign against Corona, vaccination was done by the workers of Human Services Institute in village Badara of Shahbad block.
ड्रोन से पहुँची डोज बंगलुरु के दूर-दराज ग्रामीण इलाक़ों में covid- वैक्सीन पहुँचाने के लिए ड्रोन का किया गया इ…
hi
Drones arrived by Drones Drones were used to deliver covid-vaccine to remote rural areas of Bangalore.
भारत की स्वास्थ्य सुविधाओं में क्रांतिकारी परिवर्तन साबित हो रहा आयुष्मान भारत डिजिटल हेल्थ मिशन अबतक देश के करीब करोड़…
hi
Ayushman Bharat Digital Health Mission is proving to be a revolutionary change in the health facilities of India.


#### Stanford Core NLP

In [39]:
#!pip install pycorenlp

In [15]:
from pycorenlp import StanfordCoreNLP

Before running the next chunk of code: 
1. Download the Stanford Core NLP model from [here](https://stanfordnlp.github.io/CoreNLP/#download).
2. Unizip the folder
3. cd into the folder<br>
    ```cd stanford-corenlp-4.3.2/```
4. Start the server using this command:<br>
    ```java -mx5g -cp "./*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -timeout 10000```
<br><br>

In [55]:
nlp = StanfordCoreNLP('http://localhost:9000')

def get_sentiment(text):
    res = nlp.annotate(text,
                       properties={'annotators': 'sentiment',
                                   'outputFormat': 'json',
                                   'timeout': 1000
                       })
    print(text)
    print('Sentiment:', res['sentences'][0]['sentiment'])
    print('Sentiment score:', res['sentences'][0]['sentimentValue'])
    print('Sentiment distribution (0-v. negative, 5-v. positive:', res['sentences'][0]['sentimentDistribution'])
    print('Sentiment tree (0-v. negative, 5-v. positive:', res['sentences'][0]['sentimentTree'])
    print("-"*30)

In [56]:
for tweet in data_en:
    print(get_sentiment(tweet))


bipartisan infrastructure law best roads bridges ports airports lead world st century modern cars trucks transit systems building moving
Sentiment: Positive
Sentiment score: 3
Sentiment distribution (0-v. negative, 5-v. positive: [0.01345368659542, 0.05605876671493, 0.12187630874107, 0.50010602849438, 0.30850520945421]
Sentiment tree (0-v. negative, 5-v. positive: (ROOT|sentiment=3|prob=0.500
  (NP|sentiment=3|prob=0.499 (JJ|sentiment=2|prob=0.625 bipartisan)
    (@NP|sentiment=3|prob=0.499
      (NML|sentiment=2|prob=0.800 (NN|sentiment=2|prob=0.625 infrastructure) (NN|sentiment=2|prob=0.989 law))
      (@NP|sentiment=3|prob=0.596
        (NML|sentiment=3|prob=0.645
          (NML|sentiment=3|prob=0.871 (JJS|sentiment=4|prob=0.960 best) (NNS|sentiment=2|prob=0.982 roads))
          (NNS|sentiment=2|prob=0.625 bridges))
        (NNS|sentiment=2|prob=0.625 ports))))
  (@S|sentiment=3|prob=0.665 (NP|sentiment=2|prob=0.625 airports)
    (VP|sentiment=3|prob=0.665 (VBP|sentiment=2|prob=0.9

today house passed build back better act thank speaker pelosi house leadership members worked hard voted pass bill goes united states senate look forward passing soon possible sign law
Sentiment: Negative
Sentiment score: 1
Sentiment distribution (0-v. negative, 5-v. positive: [0.13113691787607, 0.41023707927236, 0.27405335185904, 0.16935253380204, 0.0152201171905]
Sentiment tree (0-v. negative, 5-v. positive: (ROOT|sentiment=1|prob=0.410
  (PRN|sentiment=2|prob=0.468
    (NP|sentiment=2|prob=1.000 (NN|sentiment=2|prob=0.998 today) (NN|sentiment=2|prob=0.995 house))
    (VP|sentiment=1|prob=0.471 (VBD|sentiment=2|prob=0.997 passed)
      (SBAR|sentiment=1|prob=0.471
        (S|sentiment=2|prob=0.952
          (@VP|sentiment=2|prob=1.000 (VB|sentiment=2|prob=0.998 build) (PRT|sentiment=2|prob=0.998 back))
          (NP|sentiment=3|prob=0.674 (JJR|sentiment=4|prob=0.875 better) (NN|sentiment=2|prob=0.992 act)))
        (VP|sentiment=1|prob=0.471 (VBP|sentiment=3|prob=0.835 thank)
       

afternoon hosting canadian prime minister justin trudeau mexican president andrés manuel lópez obrador white house first north american leaders summit since look forward strengthening ties charting new path collaboration
Sentiment: Neutral
Sentiment score: 2
Sentiment distribution (0-v. negative, 5-v. positive: [0.10230301254497, 0.3349131601935, 0.42793797665037, 0.11663314365371, 0.01821270695744]
Sentiment tree (0-v. negative, 5-v. positive: (ROOT|sentiment=2|prob=0.428
  (NP|sentiment=2|prob=0.762 (NP|sentiment=2|prob=0.990 afternoon)
    (VP|sentiment=2|prob=0.781 (VBG|sentiment=2|prob=0.625 hosting)
      (NP|sentiment=2|prob=0.977 (JJ|sentiment=2|prob=0.625 canadian)
        (NML|sentiment=2|prob=1.000
          (NML|sentiment=2|prob=0.928 (JJ|sentiment=3|prob=0.524 prime)
            (@NML|sentiment=2|prob=0.528 (NN|sentiment=2|prob=0.625 minister) (NN|sentiment=2|prob=0.625 justin)))
          (NNP|sentiment=2|prob=0.625 trudeau)))))
  (X|sentiment=2|prob=0.547
    (X|sentimen

lot folks know car guy gotten chance drive pretty incredible vehicles years never could imagined ones like electric vehicle took spin today future electric made right america
Sentiment: Positive
Sentiment score: 3
Sentiment distribution (0-v. negative, 5-v. positive: [0.01788771747566, 0.07288869184517, 0.12670901455379, 0.49976942043471, 0.28274515569068]
Sentiment tree (0-v. negative, 5-v. positive: (ROOT|sentiment=3|prob=0.500
  (NP|sentiment=2|prob=1.000 (NN|sentiment=2|prob=0.998 lot) (NNS|sentiment=2|prob=0.998 folks))
  (VP|sentiment=3|prob=0.472 (VBP|sentiment=2|prob=0.999 know)
    (SBAR|sentiment=3|prob=0.500
      (NP|sentiment=2|prob=0.349
        (NP|sentiment=2|prob=1.000 (NN|sentiment=2|prob=0.988 car) (NN|sentiment=2|prob=0.998 guy))
        (VP|sentiment=3|prob=0.463
          (@VP|sentiment=2|prob=0.992 (VBN|sentiment=2|prob=0.995 gotten)
            (NP|sentiment=2|prob=0.998 (NN|sentiment=2|prob=0.998 chance) (NN|sentiment=2|prob=0.997 drive)))
          (NP|sentime

bipartisan infrastructure law first time ever create true national network charging stations electric vehicles means charge drive across country create thousands jobs electrical workers
Sentiment: Positive
Sentiment score: 3
Sentiment distribution (0-v. negative, 5-v. positive: [0.01758551150783, 0.11548786401584, 0.26376635297756, 0.46339353188455, 0.13976673961422]
Sentiment tree (0-v. negative, 5-v. positive: (ROOT|sentiment=3|prob=0.463
  (PRN|sentiment=2|prob=0.494
    (NP|sentiment=2|prob=0.713
      (NP|sentiment=2|prob=0.958 (JJ|sentiment=2|prob=0.625 bipartisan)
        (@NP|sentiment=2|prob=0.800 (NN|sentiment=2|prob=0.625 infrastructure) (NN|sentiment=2|prob=0.989 law)))
      (NP-TMP|sentiment=2|prob=0.939 (JJ|sentiment=2|prob=0.999 first) (NN|sentiment=2|prob=0.999 time)))
    (@S|sentiment=2|prob=0.657 (ADVP|sentiment=2|prob=0.999 ever)
      (VP|sentiment=3|prob=0.505
        (@VP|sentiment=3|prob=0.671 (VB|sentiment=3|prob=0.966 create)
          (NP|sentiment=2|prob=0.

...

join live u capitol weekly press conference discuss transformative results congress biden administration delivered first days
Sentiment: Neutral
Sentiment score: 2
Sentiment distribution (0-v. negative, 5-v. positive: [0.02062675466142, 0.17219916157954, 0.59084982427939, 0.20525793290487, 0.01106632657478]
Sentiment tree (0-v. negative, 5-v. positive: (ROOT|sentiment=2|prob=0.591
  (@VP|sentiment=2|prob=0.984 (VB|sentiment=2|prob=0.993 join)
    (NP|sentiment=2|prob=0.999 (JJ|sentiment=2|prob=0.995 live)
      (@NP|sentiment=2|prob=0.528 (NN|sentiment=2|prob=0.625 u) (NN|sentiment=2|prob=0.625 capitol))))
  (SBAR|sentiment=2|prob=0.788
    (NP|sentiment=2|prob=0.999 (JJ|sentiment=2|prob=0.625 weekly)
      (@NP|sentiment=2|prob=0.785 (NN|sentiment=2|prob=0.993 press) (NN|sentiment=2|prob=0.625 conference)))
    (VP|sentiment=2|prob=0.729 (VBP|sentiment=2|prob=0.959 discuss)
      (NP|sentiment=2|prob=0.798
        (NP|sentiment=2|prob=0.768 (JJ|sentiment=2|prob=0.625 transformative) (

tonight address joint session congress first days office hear president bold transformative actions build back better join us live pt/ et helpishere
Sentiment: Neutral
Sentiment score: 2
Sentiment distribution (0-v. negative, 5-v. positive: [0.04025876659836, 0.24697152484764, 0.49387500948507, 0.19325718417294, 0.02563751489599]
Sentiment tree (0-v. negative, 5-v. positive: (ROOT|sentiment=2|prob=0.494
  (NP|sentiment=1|prob=0.387
    (NP|sentiment=2|prob=0.998 (NN|sentiment=2|prob=0.625 tonight) (NN|sentiment=2|prob=0.997 address))
    (NP|sentiment=2|prob=0.442 (JJ|sentiment=2|prob=0.953 joint)
      (@NP|sentiment=2|prob=0.901 (NN|sentiment=2|prob=0.975 session) (NN|sentiment=2|prob=0.625 congress))))
  (@S|sentiment=2|prob=0.616
    (NP-TMP|sentiment=2|prob=0.998 (JJ|sentiment=2|prob=0.999 first) (NNS|sentiment=2|prob=0.998 days))
    (@S|sentiment=2|prob=0.707 (NP|sentiment=2|prob=0.989 office)
      (VP|sentiment=2|prob=0.714 (VBP|sentiment=2|prob=0.997 hear)
        (NP|sentime

days ago biden said help way thanks american rescue plan helpishere w/shots arms mo…
Sentiment: Neutral
Sentiment score: 2
Sentiment distribution (0-v. negative, 5-v. positive: [0.0211588393728, 0.17574784303312, 0.59819464659419, 0.19455349015436, 0.01034518084554]
Sentiment tree (0-v. negative, 5-v. positive: (ROOT|sentiment=2|prob=0.598
  (@FRAG|sentiment=2|prob=0.591
    (S|sentiment=2|prob=0.798
      (NP|sentiment=2|prob=1.000 (NP|sentiment=2|prob=0.998 days)
        (PP|sentiment=2|prob=0.520 (IN|sentiment=2|prob=0.998 ago) (NP|sentiment=2|prob=0.625 biden)))
      (VP|sentiment=2|prob=0.799 (VBD|sentiment=2|prob=0.998 said)
        (S|sentiment=2|prob=0.675
          (VP|sentiment=3|prob=0.552
            (@VP|sentiment=3|prob=0.743 (VB|sentiment=2|prob=0.999 help)
              (NP|sentiment=3|prob=0.820 (NN|sentiment=2|prob=0.999 way) (NNS|sentiment=3|prob=0.984 thanks)))
            (S|sentiment=2|prob=0.625 american))
          (NP|sentiment=2|prob=0.984
            (NML|se

hearts full joy taken historic step formal recognition armenian genocide day following congress resolution declaring barbarism committed armenians genocide
Sentiment: Positive
Sentiment score: 3
Sentiment distribution (0-v. negative, 5-v. positive: [0.08448843774354, 0.21762539353507, 0.15530550483562, 0.47266072872539, 0.06991993516038]
Sentiment tree (0-v. negative, 5-v. positive: (ROOT|sentiment=3|prob=0.473
  (NP|sentiment=4|prob=0.889 (NP|sentiment=3|prob=0.919 hearts)
    (NP|sentiment=3|prob=0.891 (JJ|sentiment=3|prob=0.965 full) (NN|sentiment=4|prob=0.853 joy)))
  (VP|sentiment=1|prob=0.421
    (@VP|sentiment=2|prob=0.999 (VBN|sentiment=1|prob=0.961 taken)
      (NP|sentiment=2|prob=0.403 (JJ|sentiment=3|prob=0.528 historic) (NN|sentiment=2|prob=0.991 step)))
    (NP|sentiment=1|prob=0.421
      (NP|sentiment=2|prob=0.513 (JJ|sentiment=2|prob=0.987 formal)
        (@NP|sentiment=2|prob=0.555 (NN|sentiment=2|prob=0.968 recognition)
          (@NP|sentiment=1|prob=0.557 (JJ|senti

TypeError: string indices must be integers

### Visualize top words for each category using word clouds

In [64]:
#!pip install wordninja
#!pip install pyspellchecker
#!pip install wordcloud

In [65]:
import re
import random
import math
from tqdm.notebook import tqdm
from collections import Counter


from wordcloud import WordCloud, ImageColorGenerator
import matplotlib.pyplot as plt 

import wordninja
from spellchecker import SpellChecker
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
stop_words = set(stopwords.words('english'))  
stop_words.add("amp")

# TO BE continued